### Diseño Evolutivo de Redes Neuronales Artificiales
### Baseline
Basado en el paper [Predicting Daily Returns of Global Stocks Indices: Neural Networks vs Support Vector Machines. Kaur, Dharni 2019](https://journaljemt.com/index.php/JEMT/article/view/30179)

Se desea reproducir los resultados del paper en ANN para el S&P 500.
Se usan parte de los datos diarios de cierre del S&P 500 (GSPC) obtenidos de Yahoo Finance.
Se calcularon los indicadores técnicos según las fórmulas del paper.

#### Importar paquetes

In [0]:
import numpy as np
import sklearn.preprocessing as skp
import tensorflow as tf

#### Parámetros de configuración
La idea es agrupar aquí los parámetros configurables.


In [0]:
p = {"local_file" : "baseline.csv",
     "url" : "https://raw.githubusercontent.com/"+
             "jmacostap/webstore/master/baseline.csv",
     "cache_dir" : "/content",
     "scaler" : skp.MinMaxScaler(),
     "test_fraction" : 0.20,  # 20% of samples for test data
     "neurons" : 9,
     "activation" : "sigmoid",  # for the hidden layer
     "output_activation" : "linear",  # for the output layer
     "optimizer" : tf.keras.optimizers.SGD(
         learning_rate=0.3,
         momentum=0.2,
         nesterov=False),
     "metrics" : ["mae","mape"],
     "loss" : "mse",  # could be mae or mse
     "epochs" : 500,  # training epochs
     "batch_size" : 100,
     "shuffle" : True,  # the training data
     "verbose" : 2,  # verbose training
}


#### Leer los datos de entrada directamente del repositorio de GitHub

In [21]:
path_to_downloaded_file = tf.keras.utils.get_file(
    p["local_file"],
    p["url"],
    cache_dir=p["cache_dir"],
    )
data = np.loadtxt(path_to_downloaded_file, skiprows=1, delimiter=",")
print(f"{data.shape} samples read")
# Extract targets
targets = np.reshape(data[:, -1], (-1,1))
print(f"Targets to use: {targets.shape}")
# Extraer los features
data = data[:, :-1]
print(f"Samples to use: {data.shape}")
# Normalizar los datos a [0,1]
data_scaler = p["scaler"]
data = data_scaler.fit_transform(data)
targets_scaler = skp.MinMaxScaler()
targets = targets_scaler.fit_transform(targets)
# Separar datos de prueba
test_fraction = np.int(len(data)*p["test_fraction"])
print(f"Test fraction: {test_fraction}")
train_targets = targets[:-test_fraction]
print(f"Train targets: {train_targets.shape}")
train_data = data[:-test_fraction]
print(f"Train samples: {train_data.shape}")
test_targets = targets[-test_fraction:]
print(f"Test targets: {test_targets.shape}")
test_data = data[-test_fraction:]
print(f"Test samples: {test_data.shape}")

(3022, 20) samples read
Targets to use: (3022, 1)
Samples to use: (3022, 19)
Test fraction: 604
Train targets: (2418, 1)
Train samples: (2418, 19)
Test targets: (604, 1)
Test samples: (604, 19)


#### Definición de funciones auxiliares

In [0]:
def create_model(input_size, neurons, X_train, y_train):

    # Create the model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            neurons,
            activation=p["activation"],
            input_shape=(input_size,),
            ),
        tf.keras.layers.Dense(
            1,
            activation=p["output_activation"],
            )
        ])
    model.compile(
        optimizer=p["optimizer"],
        metrics=p["metrics"],
        loss=p["loss"],
        )

    # Train the model
    model.fit(X_train, y_train, epochs=p["epochs"],
              batch_size=p["batch_size"],
              shuffle=p["shuffle"],
              verbose=p["verbose"],
              validation_split=p["test_fraction"],
              )
    return model

####Crear el modelo y entrenarlo

In [24]:
np.random.seed(31416)  # numpy
tf.random.set_seed(31416)  # keras

model = create_model(
    data.shape[1],
    p["neurons"],
    train_data,
    train_targets,
    )

Epoch 1/500
20/20 - 0s - loss: 0.1451 - mae: 0.2436 - mape: 153118.6562 - val_loss: 0.0037 - val_mae: 0.0532 - val_mape: 12.0585
Epoch 2/500
20/20 - 0s - loss: 0.0040 - mae: 0.0473 - mape: 148864.2656 - val_loss: 0.0017 - val_mae: 0.0336 - val_mape: 7.5928
Epoch 3/500
20/20 - 0s - loss: 0.0031 - mae: 0.0397 - mape: 158545.5938 - val_loss: 0.0013 - val_mae: 0.0289 - val_mape: 6.5404
Epoch 4/500
20/20 - 0s - loss: 0.0028 - mae: 0.0358 - mape: 157695.0000 - val_loss: 0.0016 - val_mae: 0.0313 - val_mape: 7.1004
Epoch 5/500
20/20 - 0s - loss: 0.0026 - mae: 0.0343 - mape: 164534.6094 - val_loss: 0.0014 - val_mae: 0.0296 - val_mape: 6.7443
Epoch 6/500
20/20 - 0s - loss: 0.0024 - mae: 0.0334 - mape: 162600.1562 - val_loss: 0.0012 - val_mae: 0.0273 - val_mape: 6.2399
Epoch 7/500
20/20 - 0s - loss: 0.0023 - mae: 0.0320 - mape: 167728.0000 - val_loss: 7.8130e-04 - val_mae: 0.0230 - val_mape: 5.2288
Epoch 8/500
20/20 - 0s - loss: 0.0021 - mae: 0.0305 - mape: 168772.8750 - val_loss: 8.6198e-04 - va

####Probar el modelo con los datos de prueba

In [25]:
loss_error, mae_error, mape_error = model.evaluate(test_data, test_targets)
print(f"Test error: {loss_error}, MAE: {mae_error}, MAPE: {mape_error}")

19/19 [==============================] - 0s 1ms/step - loss: 0.0013 - mae: 0.0321 - mape: 7.3333
Test error: 0.0012753813061863184, MAE: 0.03205246105790138, MAPE: 7.333277702331543
